The purpose of this note book, is to make you get used to 'zarr' 

---


# 0. What is zarr? 

## zarr is a file format for storing big data.  It divides your big data  into 'chunk' sized files and store each of your chunk in a 'zarr' directory.  The 'zarr' directory also stores dimensions, coordinates, (or attributes ), all sorts of meta data.  

Long story short for netcdf users:  it cuts your netcdf file in small pieces and store them in a directory, and that make your parallelism possible. So, without using MPI-IO, you can write/read your zarr file in parallel. (where as for netcdf file, even you use hdf5/parallel format, if you do not read/write your file from MPI job, you can not read/write your file in parallel. & Netcdf is not thread safe.

( for oceanography numerical model users: If you chose to read/write data as zarr (not in netcdf), for each mpi domain that you compute with a mpi process, it just writes your 'zarr' chunk, that may speed up overall computation time by winning IO access...)


*** Take away tips for Lustre users: 
As zarr is composed of many small files, in case you use lustre filesystem, do not forget to change the striping as 1 for the directory you use, before you starts to store your zarr file.
 `mkdir dir_for_zarr`
 `lfs setstripe -c 1 dir_for_zarr `
Then, save your zarr file in 'dir_for_zarr' ***


Link to zarr documentation https://zarr.readthedocs.io
---


---

# 1.  Set up your python environment 
call python environments to use xarray and dask, then create a dask-worker cluster (as explained in notebook *1 DASK, with HPC cluster (PBS Pro)* )

In [1]:
import dask
import xarray as xr

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=6,memory='30 gb', walltime='1:00:00')
w = cluster.scale(10)

In [4]:
from dask.distributed import Client
client=Client(cluster)
client

Client Scheduler: tcp://10.120.43.58:59577 Dashboard: http://10.120.43.58:8787/status,Cluster Workers: 10 Cores: 60 Memory: 300.00 GB


---
# 2. Read a zarr file, as xarray data set.

In [7]:
filename='/work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr'
ds =xr.open_zarr(filename)

In [6]:
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<shape=(8785,), chunksize=(1,)>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Data variables:
    SST      (time, face, j, i) float32 dask.array<shape=(8785, 13, 4320, 4320), chunksize=(1, 1, 4320, 4320)>

 data size: 8525.4 GB


Above, you see that the zarr file which one just read contains dataset of 'SST' in face,i,j,time ***dimention***.  *dtime* is a time cordinate.  *i* and *j* dimention corresponds almost like longtitute and latitude (XC and YC), *face* corresponds to one of 13 patch of earth surface. 

---

 You see 'chunksize=(1, 1, 4320, 4320)'  This is like mpi domain decomposition, that the SST is decomposed into these domain, and when needed,  dask's parallel process (workers) will read some of these decomposed chunks in their memory, or on their disk, and make computation.  In zarr language, the directory of zarr contains n files and each files are made of data of these chunk. 

i.e. if you use parallel file system, with huge data size, one should have less meta-data access.  Thus you better put enough size of data in each chunk, otherwise you'll just kill the meta-data server.  But, if you want to put your data in the cash of disk to have fast read-wrtite of your data, this chunk size should be smaller than the cash size, so that controller considers that these are the 'cachable' small files. (like GPFS...)  In anycase, before you 

---
# 3. Look into a zarr file.

zarr file, SST.zarr, is a directory, and it contains, directories which corresponds to the 'corrdinates' dtime, face, i iters, j, time, and 'SST' the Data variables.  

In [11]:
!ls -a /work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr

.  ..  dtime  face  i  iters  j  SST  time  .zattrs  .zgroup


The data variable directory 'SST' contains 114205 file, because 8785 (time) * 13(face) =114205, and each file corresponds to the data in each 'chunk'.

In [17]:
!ls -1 /work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr/SST/ |wc -l

114205


In [24]:
!ls -1 /work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr/SST/ |head

0.0.0.0
0.1.0.0
0.10.0.0
0.11.0.0
0.12.0.0
0.2.0.0
0.3.0.0
0.4.0.0
0.5.0.0
0.6.0.0
ls: write error: Broken pipe


For example, 0.1.0.0 contains, SST data for time = 0, face=1, and  i= 0-4319, and j= 0- 4319 

In [22]:
!ls -a /work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr/dtime

.  ..  0  .zarray  .zattrs


In [23]:
!cat /work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr/dtime/.zarray

{
    "chunks": [
        8785
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        8785
    ],
    "zarr_format": 2
}

You can look into .zarray file to see the encodings of zarr files.  You can reffer to encoding from following command.

In [6]:
ds.SST.encoding

{'chunks': (1, 1, 4320, 4320),
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float32'),
 'coordinates': 'dtime iters'}

In [7]:
ds.dtime.encoding

{'chunks': (8785,),
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'units': 'hours since 2011-11-15 00:00:00',
 'calendar': 'proleptic_gregorian',
 'dtype': dtype('int64')}

In [8]:
ds.iters.encoding

{'chunks': (1,),
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'dtype': dtype('int64')}

---
# 4.  Let's try to create a subset of data we just read, and write to another zarr file.  

In [9]:
dsmille=ds.isel(time=slice(0,1000))
print(dsmille)
print('\n data size: %.1f GB' %(dsmille.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 1000)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(1000,), chunksize=(1000,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<shape=(1000,), chunksize=(1,)>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 9.295e+06 9.299e+06
Data variables:
    SST      (time, face, j, i) float32 dask.array<shape=(1000, 13, 4320, 4320), chunksize=(1, 1, 4320, 4320)>

 data size: 970.4 GB


In [10]:
%time dsmille.to_zarr('/work/scratch/odakat/test.zarr', mode='w')

CPU times: user 1min 40s, sys: 5.43 s, total: 1min 45s
Wall time: 10min 54s


In [11]:
!du -hs /work/scratch/odakat/test.zarr

338G	/work/scratch/odakat/test.zarr


As you can see above, even the datasize is 970G, it use compressions (heritated from 'compressor 'encoding from orignal file 'ds' which is , 
** 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)**
so filesize it self is 338G. 

---

# 5. We can try other compression method here. 

In [12]:
from numcodecs import blosc
blosc.list_compressors()

['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib', 'zstd']

In [13]:
import zarr
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)

In [14]:
%time dsmille.to_zarr('/work/scratch/odakat/testzarr',  encoding={'SST': {'compressor': compressor}} , mode='w')

CPU times: user 3min 22s, sys: 8.16 s, total: 3min 30s
Wall time: 11min 14s


In [15]:
!du -hs /work/scratch/odakat/testzarr

319G	/work/scratch/odakat/testzarr


 Well, looks like new compressing made us win about 20 G of space, but took 1.5 min more..
 
 ---
# 6. clean up


In [16]:
!rm -rf /work/scratch/odakat/testzarr
!rm -rf /work/scratch/odakat/test.zarr

In [17]:
cluster.close()